# Demo - Applications of MultiAttack (CIFAR10)

In [1]:
import os
import sys
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import torchvision.utils
from torchvision import models
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torchattacks

from models import Target

In [2]:
batch_size = 128

cifar10_test  = dsets.CIFAR10(root='./data', train=False,
                              download=True, transform=transforms.ToTensor())

test_loader = torch.utils.data.DataLoader(cifar10_test,
                                          batch_size=batch_size,
                                          shuffle=False)

Files already downloaded and verified


In [3]:
model = Target().cuda()
model.load_state_dict(torch.load("./checkpoints/target.pth"))
model = model.eval()

## 1. PGD Random Restart

In [4]:
pgd = torchattacks.PGD(model, eps=4/255, alpha=2/255, steps=4, random_start=True)
atk = torchattacks.MultiAttack([pgd]*10)

In [5]:
# Number of Random Restart = 1
pgd.save(data_loader=test_loader, save_path=None, verbose=True)

In [6]:
# Number of Random Restart = 10
atk.save(data_loader=test_loader, save_path=None, verbose=True)

## 2. CW c-search

In [7]:
c_candidate = [1, 0.1, 0.01, 0.001]
atk = torchattacks.MultiAttack([torchattacks.CW(model, c, kappa=0, steps=1000, lr=0.01) for c in c_candidate])

In [8]:
# Find the best c in c_candidate
atk.save(data_loader=test_loader, save_path=None, verbose=True)

## 3. Attack on Only Correct Examples

Note, the accuracy of adversarial attack on only correct examples is usually same as the accuracy of advesarial attack on whole examples except in following cases:
    * Using Random Start (below)
    * Catstrophic Overfitting (https://arxiv.org/abs/2010.01799)

In [9]:
import torchattacks

In [10]:
vanila = torchattacks.VANILA(model)
pgd = torchattacks.PGD(model, eps=8/255, alpha=2/255, steps=4, random_start=True)

In [11]:
atk = torchattacks.MultiAttack([vanila, pgd])

In [12]:
# Clean Accuracy
pgd.save(data_loader=test_loader, save_path=None, verbose=True)

In [13]:
# Robust Accuracy on only Correct Examples
atk.save(data_loader=test_loader, save_path=None, verbose=True)